# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 12:42PM,253002,10,8813601,MedStone Pharma LLC,Released
1,Dec 13 2022 12:42PM,253002,10,8813608,MedStone Pharma LLC,Released
2,Dec 13 2022 12:42PM,253002,10,8813609,MedStone Pharma LLC,Released
3,Dec 13 2022 12:42PM,253002,10,8813610,MedStone Pharma LLC,Released
4,Dec 13 2022 12:42PM,253002,10,8813611,MedStone Pharma LLC,Released
5,Dec 13 2022 12:42PM,253002,10,8813612,MedStone Pharma LLC,Released
6,Dec 13 2022 12:42PM,253002,10,8813613,MedStone Pharma LLC,Released
7,Dec 13 2022 12:42PM,253002,10,8838584,MedStone Pharma LLC,Released
8,Dec 13 2022 12:42PM,253002,10,8838585,MedStone Pharma LLC,Released
9,Dec 13 2022 12:42PM,253002,10,8838587,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,252997,Released,2
45,252999,Released,1
46,253000,Released,8
47,253001,Released,9
48,253002,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252997,NaN,NaN,2.0
252999,NaN,NaN,1.0
253000,NaN,NaN,8.0
253001,NaN,NaN,9.0
253002,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0.0,0.0,1.0
252861,0.0,4.0,1.0
252891,1.0,0.0,4.0
252899,0.0,0.0,1.0
252927,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0,0,1
252861,0,4,1
252891,1,0,4
252899,0,0,1
252927,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,0,0,1
1,252861,0,4,1
2,252891,1,0,4
3,252899,0,0,1
4,252927,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,,,1
1,252861,,4,1
2,252891,1,,4
3,252899,,,1
4,252927,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC
13,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC
22,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC
30,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations"
31,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC"
33,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC"
36,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation
52,Dec 13 2022 12:17PM,252995,10,Emerginnova
55,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation
59,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,,,13
1,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,,,9
2,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,,,8
3,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",,,1
4,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",,,2
5,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC",,,3
6,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation,,,16
7,Dec 13 2022 12:17PM,252995,10,Emerginnova,,,3
8,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,,,4
9,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13,,
1,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9,,
2,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,8,,
3,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",1,,
4,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",2,,
5,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC",3,,
6,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation,16,,
7,Dec 13 2022 12:17PM,252995,10,Emerginnova,3,,
8,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,4,,
9,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13,,
1,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9,,
2,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,8,,
3,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",1,,
4,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13.0,NaN,NaN
1,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9.0,NaN,NaN
2,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,8.0,NaN,NaN
3,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
4,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13.0,0.0,0.0
1,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9.0,0.0,0.0
2,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,8.0,0.0,0.0
3,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
4,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5312318,132.0,64.0,1.0
102,252987,0.0,1.0,0.0
12,505931,4.0,0.0,0.0
15,758799,86.0,30.0,47.0
16,1011787,3.0,4.0,0.0
17,252964,1.0,0.0,0.0
19,505845,3.0,0.0,0.0
20,758946,3.0,0.0,0.0
21,505971,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5312318,132.0,64.0,1.0
1,102,252987,0.0,1.0,0.0
2,12,505931,4.0,0.0,0.0
3,15,758799,86.0,30.0,47.0
4,16,1011787,3.0,4.0,0.0
5,17,252964,1.0,0.0,0.0
6,19,505845,3.0,0.0,0.0
7,20,758946,3.0,0.0,0.0
8,21,505971,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,132.0,64.0,1.0
1,102,0.0,1.0,0.0
2,12,4.0,0.0,0.0
3,15,86.0,30.0,47.0
4,16,3.0,4.0,0.0
5,17,1.0,0.0,0.0
6,19,3.0,0.0,0.0
7,20,3.0,0.0,0.0
8,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,132.0
1,102,Released,0.0
2,12,Released,4.0
3,15,Released,86.0
4,16,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21
Status,,,,,,,,,
Completed,1.0,0.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
Executing,64.0,1.0,0.0,30.0,4.0,0.0,0.0,0.0,0.0
Released,132.0,0.0,4.0,86.0,3.0,1.0,3.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21
0,Completed,1.0,0.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
1,Executing,64.0,1.0,0.0,30.0,4.0,0.0,0.0,0.0,0.0
2,Released,132.0,0.0,4.0,86.0,3.0,1.0,3.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21
0,Completed,1.0,0.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
1,Executing,64.0,1.0,0.0,30.0,4.0,0.0,0.0,0.0,0.0
2,Released,132.0,0.0,4.0,86.0,3.0,1.0,3.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()